In [ ]:
import nglview as nv
import numpy as np

In [ ]:
def save_pdb(coordinates, atom_types, output_file):
    """
    Save atomic coordinates and types to a PDB file.

    Parameters:
        coordinates (np.ndarray): N x 3 array of atomic coordinates.
        atom_types (list of str): List of atom types corresponding to each coordinate.
        output_file (str): Path to save the .pdb file.
    """
    with open(output_file, 'w') as f:
        f.write("HEADER    Generated by Model\n")
        for i, (atom_type, coord) in enumerate(zip(atom_types, coordinates), start=1):
            x, y, z = coord
            f.write(
                f"ATOM  {i:5d}  {atom_type:<4} MOL     1    {x:8.3f}{y:8.3f}{z:8.3f}  1.00  0.00           {atom_type[0]:>2}\n"
            )
        f.write("END\n")

# Example Data
coordinates = np.array([[0.0, 0.0, 0.0], [1.5, 0.0, 0.0], [0.0, 1.5, 0.0]])
atom_types = ['C', 'O', 'N']
pdb_file = "model_sample.pdb"

# Save the PDB file
save_pdb(coordinates, atom_types, pdb_file)

# Visualize in Jupyter Notebook
view = nv.show_file(pdb_file)  # Load the PDB file
view.add_representation('ball+stick')  # Add visualization style
view.display()
